In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from copy import copy
from pprint import pprint
from plot import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import itertools
import abc
import sys
import re

sys.path.append("../")
import gpusims
import gpusims.plot.metrics as metric
from gpusims.plot.data import PlotData
from gpusims.config import Config, parse_configs
from gpusims.bench import parse_benchmarks

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
root_dir = Path("/Users/roman/dev/gpgpusims")
benchmark_dir = root_dir / "benchmarks"
run_dir = root_dir / "run"
assert benchmark_dir.is_dir()
assert run_dir.is_dir()

In [5]:
simulators = copy(gpusims.SIMULATORS)
configs = parse_configs(benchmark_dir / "configs" / "configs.yml")
benchmarks = parse_benchmarks(benchmark_dir / "benchmarks.yml")

pprint(simulators)
pprint(configs)
pprint(benchmarks)

{'accelsim-ptx': <class 'gpusims.accelsim.AccelSimPTXBenchmarkConfig'>,
 'accelsim-sass': <class 'gpusims.accelsim_sass.AccelSimSASSBenchmarkConfig'>,
 'm2s': <class 'gpusims.multi2sim.Multi2SimBenchmarkConfig'>,
 'macsim': <class 'gpusims.macsim.MacSimBenchmarkConfig'>,
 'native': <class 'gpusims.native.NativeBenchmarkConfig'>,
 'tejas': <class 'gpusims.tejas.TejasBenchmarkConfig'>}
{'sm6_gtx1080': Config(key='sm6_gtx1080', name='GTX 1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 20, 'clock_speed': 1607}),
 'sm86_a4000': Config(key='sm86_a4000', name='A4000', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_A4000'), spec={'sm_count': 48, 'clock_speed': 735}),
 'sm86_rtx3070': Config(key='sm86_rtx3070', name='RTX 3070', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_RTX3070'), spec={'sm_count': 46, 'clock_speed': 1132})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStrea

In [6]:
def plot_scatter(
    data, config, metric_cls,
    title=None, fontsize=32, font_family="Helvetica", round_to=2
):
    traces = []
    data = data.set_index(["Simulator"])
    data = data.sort_values(by="Benchmark")
    data = data.sort_index()
    simulators = data.index.get_level_values("Simulator").unique().tolist()
    all_data = []
    
    for sim in simulators:
        if sim == gpusims.NATIVE:
            continue
        
        sim_values = data.loc[data.index == sim] # .reset_index()
        sim_values = sim_values.rename(columns={"Value": "SimValue"})
        
        hw_values = data.loc[data.index == gpusims.NATIVE] # .reset_index()
        hw_values = hw_values.rename(columns={"Value": "HwValue"})
        values = sim_values.merge(hw_values, on="Benchmark")
        
        hw_values = values["HwValue"].to_numpy()
        sim_values = values["SimValue"].to_numpy()
            
        # print(sim, "hw", hw_values)
        # print(sim, "sim", sim_values)
        all_data.append(values)
        
        if sim_values.sum() > 0:
            correl_co = np.corrcoef(hw_values, sim_values)[0][1]
        else:
            correl_co = 0
        errs = sim_values - hw_values
        
        rel_errs = np.absolute(errs) / (hw_values+0.0000001)
        assert rel_errs.shape == errs.shape
        
        assert len(errs) > 0
        mean_total_err = np.absolute(rel_errs).sum() / len(errs)
        
        assert hw_values.sum() > 0
        mean_total_agg_err = np.absolute(errs).sum() / hw_values.sum()
        mae = np.absolute(errs).sum() / len(errs)
        
        assert len(hw_values) > 0
        mse = np.power(errs, 2).sum() / len(hw_values)
        nrmse = np.power(errs, 2).sum() / np.power(hw_values, 2).sum()

        traces.append(go.Scatter(
            x = values["HwValue"],
            y = values["SimValue"],
            hovertext = values["Benchmark"],
            mode = 'markers',
            marker = dict(
                size = 16,
                color = "rgba(%d, %d, %d, %f)" % (*hex_to_rgb(SIM_COLOR[sim]), 0.7),
                symbol = "x",
            ),
            #error_x=dict(
            #    type='data',
            #    symmetric=False,
            #    array=hw_error,
            #    arrayminus=hw_error_min,
            #    visible=True
            #),
            name="{}<br>Corr={:.3f}, Err={:.1f}%, NRMSE={:.1f}%<br>".format(
                SIM_NAME[sim], correl_co, mean_total_err*100, nrmse*100),
        ))

    all_data = pd.concat(all_data, axis=0)
    all_values = all_data[["HwValue", "SimValue"]]
    min_ax_val = all_values.min().min()
    max_ax_val = all_values.max().max()
    
    pad = 0.1
    real_x_range = [min_ax_val, max_ax_val]
    
    typ = "log"
    # typ = "linear"
    if typ == "log":
        # closest power of 10
        if min_ax_val > 0:
            x_range_min = np.log10(min_ax_val)
        else:
            x_range_min = min_ax_val
        assert max_ax_val > 0
        x_range = [np.floor(x_range_min), np.log10(max_ax_val) * (1+pad)]
    else:
        x_range = [min_ax_val, max_ax_val * (1+pad)]
    
    # print(x_range)
    
    gridcolor = "rgba(128,128,128,.4)"
    annotations = []
    if False:
        annotations = [
            go.layout.Annotation(
                text=sim,
                xref='paper', # use paper coordinates
                yref='paper', # use paper coordinates
                x=0.01 + i * 0.15, # position in norm. coord
                y=1.20, # position in norm. coord  
                xanchor="center",
                font=go.layout.annotation.Font(
                    size=fontsize,
                    color='Black'
                ),
                showarrow=False,
                bgcolor=sim_color[sim],
                # bordercolor='#FFFFFF',
                borderwidth=0,
                borderpad=6,
            ) for i, sim in enumerate(simulators)
        ]
    layout = go.Layout(
        title=dict(
            text=title,
            x=0.5,
            y=0.98,
            xanchor="center",
            yanchor="top",
        ),
        font_family="Helvetica",
        font_color="black",
        font_size=fontsize,
        xaxis=dict(
            title=f"Hardware {metric_cls.name}",
            gridcolor=gridcolor,
            zerolinecolor=gridcolor,
            type=typ,
            range=x_range,
        ),
        yaxis=dict(
            title=f"Simulation {metric_cls.name}",
            gridcolor=gridcolor,
            zerolinecolor=gridcolor,
            type=typ,
            range=x_range,
        ),
        annotations=annotations,
        **DEFAULT_LAYOUT_OPTIONS,
    )
    layout.width = 1450
    layout.height = 850
    layout.margin = go.layout.Margin(
        pad=10,
        autoexpand=True,
        l=MARGIN, r=MARGIN, t=2*MARGIN, b=MARGIN
    )
    
    if True:
        xyrange = np.array([0, layout.xaxis.range[1]])
        if typ == "log":
            xyrange = np.power(xyrange, 10)
        xyline = go.Scatter(
            x=xyrange,
            y=xyrange,
            showlegend=False,
            mode="lines",
            line=dict(
                # color='rgba(255,0,0,.7)',
                color="black",
            ),
        )
        traces.append(xyline)
        
    fig = go.Figure(data=traces, layout=layout)
    return fig

In [7]:
def correl_err_table(err_data):
    # pprint(table_data)
    sim_line = [""]
    conf_line = [""]
    for si, sim in enumerate([s for s in selected_simulators_copy if s != gpusims.NATIVE]):
        sim_line.append("\multicolumn{2}{c|}{%s}" % SIM_NAME_TEX[sim])
        conf_line += [r"\scriptsize \centering\arraybackslash %s" % configs[c].name for c in plot_configs]
    print("\n & ".join(sim_line) + r" \\")
    print("%")
    print("\n & ".join(conf_line) + r" \\ \hline")
    print("%")
    for metric_key, metric_name in [("corr", "Corr."), ("err", "Rel. Err"), ("nrmse", "NRMSE")]:
        line = [metric_name]
        for si, sim in enumerate([s for s in selected_simulators_copy if s != gpusims.NATIVE]):
            # if sim == gpusims.NATIVE:
            #     continue
            for ci, conf in enumerate(plot_configs):
                matches = [e for e in err_data if e["config"] == conf and e["sim"] == sim]
                value = ""
                if len(matches) == 1:
                    value = matches[0][metric_key]
                line.append(value)
        print(" & ".join(line) + r" \\")
        print("%")

# correl_err_table(table_data)

In [8]:
def plot_scatter_subplots(
    in_data, metric_cls,
    title=None, fontsize=32, font_family="Helvetica", round_to=2
):
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=["GTX 1080", "RTX 3070"],
                        horizontal_spacing=0.15)
    
    data = in_data.set_index(["Simulator", "Config"])
    data = data.sort_values(by=["Config", "Benchmark"])
    data = data.sort_index()
    simulators = data.index.get_level_values("Simulator").unique().tolist()
    # configs = data.index.get_level_values("Config").unique().tolist()
    plot_configs = ["sm6_gtx1080", "sm86_rtx3070"]
    
    all_data = []
    annotations = []
    table_data = []
    
    for ci, conf in enumerate(plot_configs):
        print(conf)
        vsi = 0
        for si, sim in enumerate(simulators):
            if sim == gpusims.NATIVE:
                continue
            vsi += 1

            sim_values = data.loc[data.index == (sim, conf)] # .reset_index()
            sim_values = sim_values.rename(columns={"Value": "SimValue"})

            hw_values = data.loc[data.index == (gpusims.NATIVE, conf)] # .reset_index()
            hw_values = hw_values.rename(columns={"Value": "HwValue"})
            
            # print(sim_values)
            # print(hw_values)
            values = sim_values.merge(hw_values, on="Benchmark")

            hw_values = values["HwValue"].to_numpy()
            sim_values = values["SimValue"].to_numpy()

            # print(sim, "hw", hw_values)
            # print(sim, "sim", sim_values)
            
            all_data.append(values)

            if sim_values.sum() > 0:
                correl_co = np.corrcoef(hw_values, sim_values)[0][1]
            else:
                correl_co = 0
            errs = sim_values - hw_values

            rel_errs = np.absolute(errs) / (hw_values+0.0000001)
            assert rel_errs.shape == errs.shape

            assert len(errs) > 0
            mean_total_err = np.absolute(rel_errs).sum() / len(errs)

            assert hw_values.sum() > 0
            mean_total_agg_err = np.absolute(errs).sum() / hw_values.sum()
            mae = np.absolute(errs).sum() / len(errs)

            assert len(hw_values) > 0
            mse = np.power(errs, 2).sum() / len(hw_values)
            nrmse = np.power(errs, 2).sum() / np.power(hw_values, 2).sum()

            fig.add_trace(go.Scatter(
                x = values["HwValue"],
                y = values["SimValue"],
                hovertext = values["Benchmark"],
                mode = 'markers',
                marker = dict(
                    size = 16,
                    color = "rgba(%d, %d, %d, %f)" % (*hex_to_rgb(SIM_COLOR[sim]), 0.7),
                    symbol = "x",
                ),
                name = SIM_NAME[sim],
                showlegend=ci==0,
                # name="{}<br>Corr={:.3f}, Err={:.1f}%, NRMSE={:.1f}%<br>".format(
                #     SIM_NAME[sim], correl_co, mean_total_err*100, nrmse*100),
            ), row=1, col=ci+1)
            
            rel_err = mean_total_err*100
            table_data.append(dict(
                config=conf,
                sim=sim,
                corr="{:.3f}".format(correl_co),
                err=r"{:.1f}\%".format(rel_err) if rel_err < 100.0 else r"{:.0f}\%".format(rel_err),
                nrmse=r"{:.1f}\%".format(nrmse*100) if nrmse*100 < 100.0 else r"{:.0f}\%".format(nrmse*100),
            ))
            anno_text = "{}<br>Corr={:.3f}, Err={:.1f}%, NRMSE={:.1f}%<br>".format(
                SIM_NAME[sim], correl_co, mean_total_err*100, nrmse*100
            )
            base_frac = 200 / 600
            frac = 1.0 / (len(simulators)-1)
            annotations.append(
                go.layout.Annotation(
                    text=anno_text,
                    xref='paper', # use paper coordinates
                    yref='paper', # use paper coordinates
                    x=0.0 + ci*0.5, # + i * 0.15, # position in norm. coord
                    # y=1.5 - (si * 0.1), # position in norm. coord 
                    y=1.1 + (vsi * base_frac * frac), # position in norm. coord 
                    # y=300 + (si * 40), # position in norm. coord 
                    # y=0 + (si * 10),
                    # xanchor="center",
                    xanchor="left",
                    yanchor="top",
                    font=go.layout.annotation.Font(
                        size=0.5 * fontsize,
                        color='Black'
                    ),
                    showarrow=False,
                    bgcolor=SIM_COLOR[sim],
                    # bordercolor='#FFFFFF',
                    borderwidth=0,
                    borderpad=6,
                ))

    all_data = pd.concat(all_data, axis=0)
    all_values = all_data[["HwValue", "SimValue"]]
    min_ax_val = all_values.min().min()
    max_ax_val = all_values.max().max()
    
    pad = 0.1
    real_x_range = [min_ax_val, max_ax_val]
    
    typ = "log"
    # typ = "linear"
    if typ == "log":
        # closest power of 10
        if min_ax_val > 0:
            x_range_min = np.log10(min_ax_val)
        else:
            x_range_min = min_ax_val
        assert max_ax_val > 0
        x_range = [np.floor(x_range_min), np.log10(max_ax_val) * (1+pad)]
    else:
        x_range = [min_ax_val, max_ax_val * (1+pad)]
    
    # print(x_range)
    
    gridcolor = "rgba(128,128,128,.4)"
    xaxis=dict(
        title=f"Hardware {metric_cls.name}",
        gridcolor=gridcolor,
        zerolinecolor=gridcolor,
        type=typ,
        range=x_range,
    )
    yaxis=dict(
        # title=f"Simulation {metric_cls.name}",
        gridcolor=gridcolor,
        zerolinecolor=gridcolor,
        type=typ,
        range=x_range,
    )
    
    # pprint(annotations)
    fig.update_layout(
        **DEFAULT_LAYOUT_OPTIONS,
    )
    fig.update_layout(
        height=600,
        width=1500,
        # annotations=annotations,
        # title_text=title,
        # title_x=0.5,
        # title_y=0.98,
        # title_xanchor="center",
        # title_yanchor="top",
        font_family="Helvetica",
        font_color="black",
        font_size=fontsize,
        xaxis=xaxis,
        xaxis2=xaxis,
        yaxis=yaxis,
        yaxis2=yaxis,
        margin = go.layout.Margin(
            pad=10,
            autoexpand=True,
            l=MARGIN, r=MARGIN, t=MARGIN, b=MARGIN
        )
    )
    fig.update_layout(
        # xaxis_title=f"Hardware {metric_cls.name}",
        yaxis_title=f"Simulation {metric_cls.name}",
    )
    for anno in fig.layout.annotations:
        anno.font = go.layout.annotation.Font(size = fontsize)
    # print(fig.layout)
    
    for ci, conf in enumerate(plot_configs):
        xyrange = np.array([0, x_range[1]])
        if typ == "log":
            xyrange = np.power(xyrange, 10)
        fig.add_trace(go.Scatter(
            x=xyrange,
            y=xyrange,
            showlegend=False,
            mode="lines",
            line=dict(
                # color='rgba(255,0,0,.7)',
                color="black",
            ),
        ), row=1, col=ci+1)
    
    correl_err_table(table_data)
    
    return fig

In [11]:
metrics = {
    gpusims.plot.metrics.Cycles: plot_scatter,
    gpusims.plot.metrics.L2Accesses: plot_scatter,
    gpusims.plot.metrics.L2Reads: plot_scatter,
    gpusims.plot.metrics.L2Writes: plot_scatter,
    gpusims.plot.metrics.L2ReadHit: plot_scatter,
    gpusims.plot.metrics.L2WriteHit: plot_scatter,
    gpusims.plot.metrics.DRAMReads: plot_scatter,
    gpusims.plot.metrics.DRAMWrites: plot_scatter,
    gpusims.plot.metrics.IPC: plot_scatter,
    gpusims.plot.metrics.InstructionCount: plot_scatter,
}
if False:
    metrics = {
        gpusims.plot.metrics.Cycles: plot_scatter,
    }

selected_configs_copy = selected_configs.copy()
selected_benchmarks_copy = selected_benchmarks.copy()
selected_simulators_copy = selected_simulators.copy()

for metric_cls, metrics_plot_func in metrics.items():
    print(metric_cls.name)
    all_metric_df = []
    # for (config_name, config) in selected_configs_copy.items():
    for config_name in selected_configs_copy:
        config = configs[config_name]
        per_config_metric_df = []
        
        # for (bench_name, bench) in selected_benchmarks_copy.items():
        for bench_name, selected_bench_inputs in selected_benchmarks_copy:
            bench = benchmarks[bench_name]
            supported_simulators = [
                sim_name for sim_name in selected_simulators
                if bench.enabled(sim_name) # and inp.enabled(sim_name)
            ]
            # for inp in bench.inputs:
            for inp_args, inp_abbr in selected_bench_inputs:
                # print(config_name, bench_name, inp)
                
                inp = next(i for i in bench.inputs if i.args.strip() == inp_args.strip())
                assert inp is not None, f"input {inp_args} does not exist"
                
                plot_data = PlotData(benchmark=bench, config=config, inp=inp)
                # for (sim_name, sim) in selected_simulators_copy.items():
                for sim_name in supported_simulators:
                    sim = simulators[sim_name]
                    
                    if not bench.enabled(sim_name):
                        continue
                    if not inp.enabled(sim_name):
                        continue
                    # print(sim_name, config_name, bench_name)
                    bench_config = sim(
                        run_dir=run_dir / sim_name.lower(),
                        benchmark=bench,
                        config=config,
                    )
                    if not bench_config.input_path(inp).is_dir():
                        print(f"WARN: {bench_config.input_path(inp)} does not exist")
                        continue

                    plot_data[sim_name] = bench_config.load_dataframe(inp)

                metric = metric_cls(plot_data)
                metric_df = metric.compute()
                metric_df["Benchmark"] = f"{bench.name}<br>{inp.args}"
                metric_df["Config"] = config_name

                per_config_metric_df.append(metric_df)
                all_metric_df.append(metric_df)

        per_config_metric_df = pd.concat(per_config_metric_df)
        # break
        # continue
        
        fig = metrics_plot_func(
            data=per_config_metric_df,
            config=config,
            metric_cls=metric_cls,
            title=f"{metric_cls.name} Correlation ({config.name})",
        )
        filename = ["scatter", metric_cls.name, config.key]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, **PDF_OPTS)
        print("wrote", filename)
    
    all_metric_df = pd.concat(all_metric_df)
    if True:
        fig = plot_scatter_subplots(
            in_data=all_metric_df,
            metric_cls=metric_cls,
            title=f"{metric_cls.name} Correlation",
        )
        filename = ["per-configs-scatter", metric_cls.name]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, **PDF_OPTS)
        print("wrote", filename)
        
    fig = metrics_plot_func(
        data=all_metric_df,
        config=config,
        metric_cls=metric_cls,
        title=f"{metric_cls.name} Correlation",
    )
    filename = ["all-configs-scatter", metric_cls.name]
    filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
    filename = filename.with_suffix(".pdf")
    fig.write_image(filename, **PDF_OPTS)
    print("wrote", filename)

# all_metric_df

Cycles
wrote figs/scatter_cycles_sm6_gtx1080.pdf
wrote figs/scatter_cycles_sm86_a4000.pdf
wrote figs/scatter_cycles_sm86_rtx3070.pdf
sm6_gtx1080
sm86_rtx3070

 & \multicolumn{2}{c|}{\textsc{GpuTejas}}
 & \multicolumn{2}{c|}{\textsc{MacSim}}
 & \multicolumn{2}{c|}{\textsc{Multi2Sim}}
 & \multicolumn{2}{c|}{\textsc{AccelSim PTX}}
 & \multicolumn{2}{c|}{\textsc{AccelSim SASS}} \\
%

 & \scriptsize \centering\arraybackslash GTX 1080
 & \scriptsize \centering\arraybackslash RTX 3070
 & \scriptsize \centering\arraybackslash GTX 1080
 & \scriptsize \centering\arraybackslash RTX 3070
 & \scriptsize \centering\arraybackslash GTX 1080
 & \scriptsize \centering\arraybackslash RTX 3070
 & \scriptsize \centering\arraybackslash GTX 1080
 & \scriptsize \centering\arraybackslash RTX 3070
 & \scriptsize \centering\arraybackslash GTX 1080
 & \scriptsize \centering\arraybackslash RTX 3070 \\ \hline
%
Corr. & 0.866 & 0.818 & 0.956 & 0.941 & -0.145 & -0.279 & 0.964 & 0.975 & 0.960 & 0.970 \\
%
Rel. Err & 8